In [4]:
import pandas as pd
import os
import yt_dlp
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import webbrowser
from threading import Thread

# Global variables to control the stop action and session counters
stop_download = False
session_video_count = 0
session_images_count = 0
session_text_count = 0

# Create directories for each platform and file type
platforms = ['tiktok', 'youtube', 'facebook', 'instagram', 'twitter']
file_types = ['videos', 'text', 'images']

for platform in platforms:
    for file_type in file_types:
        os.makedirs(f'downloads/{platform}/{file_type}', exist_ok=True)

def update_counters():
    global session_video_count, session_images_count, session_text_count
    video_counter_label.config(text=f"Session Videos: {session_video_count}")
    images_counter_label.config(text=f"Session Images: {session_images_count}")
    text_counter_label.config(text=f"Session Text: {session_text_count}")

def download_video(url, platform):
    global stop_download, session_video_count, session_images_count, session_text_count
    if stop_download:
        print("Download stopped.")
        return

    ydl_opts = {
        'quiet': False,
        'no_warnings': True,
        'format': 'mp4'  # Ensure only mp4 format is downloaded
    }

    if platform == 'instagram':
        ydl_opts['cookiefile'] = r'C:\Users\mesha\OneDrive\Desktop\proj\www.instagram.com_cookies.txt'  # Ensure the path to your cookies file is correct

    if platform == 'twitter':
        ydl_opts['cookiefile'] = r'C:\Users\mesha\OneDrive\Desktop\proj\x.com_cookies'  # Ensure the path to your cookies file is correct

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=False)
            
            # Check for video formats
            formats = info_dict.get('formats', [])
            if not formats:
                raise Exception("No video formats found.")
            
            # Determine the type of file
            file_type = 'videos'
            ext = info_dict.get('ext', '')
            
            if ext in ['jpg', 'jpeg', 'png', 'gif']:
                file_type = 'images'
            elif ext in ['txt', 'srt', 'json', 'xml', 'pdf']:
                file_type = 'text'
            
            # Set output template based on file type
            ydl_opts['outtmpl'] = f'downloads/{platform}/{file_type}/%(title)s.%(ext)s'
            ydl = yt_dlp.YoutubeDL(ydl_opts)
            ydl.download([url])
            
        print(f"Downloaded {file_type} from: {url}")
        content_text.insert(tk.END, f"Downloaded {file_type} from: {url}\n")
        content_text.see(tk.END)  # Auto-scroll to the end
        
        # Update session counters
        if file_type == 'videos':
            session_video_count += 1
        elif file_type == 'images':
            session_images_count += 1
        elif file_type == 'text':
            session_text_count += 1
        
        update_counters()
    except Exception as e:
        print(f"Failed to download from: {url}, error: {e}")
        content_text.insert(tk.END, f"Failed to download from: {url}, error: {e}\n")
        content_text.see(tk.END)

def get_platform(url):
    if 'tiktok.com' in url:
        return 'tiktok'
    elif 'youtube.com' in url or 'youtu.be' in url:
        return 'youtube'
    elif 'facebook.com' in url or 'fb.watch' in url:
        return 'facebook'
    elif 'instagram.com' in url:
        return 'instagram'
    elif 'twitter.com' in url:
        return 'twitter'
    else:
        return 'unknown'

def start_download(file_path):
    global stop_download, session_video_count, session_images_count, session_text_count
    stop_download = False
    session_video_count = 0
    session_images_count = 0
    session_text_count = 0
    update_counters()
    content_text.delete('1.0', tk.END)
    
    try:
        df = pd.read_excel(file_path)
        print("Excel file loaded successfully.")
        print("First few rows of the Excel file:")
        print(df.head())
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        messagebox.showerror("Error", f"Error loading Excel file: {e}")
        return

    # Ensure we only process the 'url' column if it exists
    if 'url' in df.columns:
        urls = df['url']
    else:
        print("No 'url' column found in the Excel file.")
        messagebox.showerror("Error", "No 'url' column found in the Excel file.")
        return

    total_links_counter_label.config(text=f"Total Links: {len(urls)}")

    # Iterate over each URL in the 'url' column and download videos
    print("Starting video download process...")
    for url in urls:
        if stop_download:
            break
        if pd.notna(url):
            platform = get_platform(url)
            print(f"Detected {platform} media from {url}")
            content_text.insert(tk.END, f"Detected {platform} media from {url}\n")
            content_text.see(tk.END)
            if platform in platforms:
                download_video(url, platform)
            else:
                print(f"Unknown platform for URL: {url}")
                content_text.insert(tk.END, f"Unknown platform for URL: {url}\n")
                content_text.see(tk.END)
    
    print("Video download process completed.")
    messagebox.showinfo("Info", "Video download process completed.")

def stop_downloads():
    global stop_download
    stop_download = True
    messagebox.showinfo("Download Stopped", f"Download process stopped.\nVideos: {session_video_count}\nImages: {session_images_count}\nText: {session_text_count}")

def select_file():
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        file_entry.delete(0, tk.END)
        file_entry.insert(0, file_path)

def open_download_folder(folder_type):
    folder_path = os.path.abspath(f'downloads')
    platform_folders = [os.path.join(folder_path, platform, folder_type) for platform in platforms]
    for platform_folder in platform_folders:
        webbrowser.open(f'file:///{platform_folder}')

def create_folder_icon(canvas, x, y):
    canvas.create_rectangle(x, y, x+50, y+40, fill="#3498db", outline="black")
    canvas.create_polygon(x+10, y, x+20, y-10, x+40, y-10, x+50, y, fill="#3498db", outline="black")

def on_enter(e):
    e.widget['background'] = '#2ecc71'

def on_leave(e):
    e.widget['background'] = '#27ae60'

# Create the main window
root = tk.Tk()
root.title('Excel Download Manager')
root.configure(background='#ecf0f1')

# Create a frame for the toolbar area
toolbar_frame = ttk.Frame(root)
toolbar_frame.pack(side='top', fill='x')

# Add buttons for Excel-like toolbar/menu
select_button = tk.Button(toolbar_frame, text='Select Excel File', command=select_file, bg='#27ae60', fg='white', relief='flat')
select_button.pack(side='left', padx=2)
select_button.bind("<Enter>", on_enter)
select_button.bind("<Leave>", on_leave)

start_button = tk.Button(toolbar_frame, text='Start Download', command=lambda: Thread(target=start_download, args=(file_entry.get(),)).start(), bg='#27ae60', fg='white', relief='flat')
start_button.pack(side='left', padx=2)
start_button.bind("<Enter>", on_enter)
start_button.bind("<Leave>", on_leave)

stop_button = tk.Button(toolbar_frame, text='Stop Download', command=stop_downloads, bg='#e74c3c', fg='white', relief='flat')
stop_button.pack(side='left', padx=2)
stop_button.bind("<Enter>", lambda e: e.widget.config(background='#c0392b'))
stop_button.bind("<Leave>", lambda e: e.widget.config(background='#e74c3c'))

# Entry to display selected file path
file_entry = ttk.Entry(toolbar_frame, width=50)
file_entry.pack(side='left', padx=2)

# Add a text widget to display the download process
content_text = tk.Text(root, height=20, wrap='none', bg='#fdfefe', fg='#2c3e50', relief='flat')
content_text.pack(side='top', fill='x', padx=10, pady=10)

# Create frames for folder sections
video_frame = ttk.LabelFrame(root, text='Videos', padding=(10, 10))
images_frame = ttk.LabelFrame(root, text='Images', padding=(10, 10))
text_frame = ttk.LabelFrame(root, text='Text', padding=(10, 10))

video_frame.pack(side='left', fill='both', expand=True, padx=5, pady=5)
images_frame.pack(side='left', fill='both', expand=True, padx=5, pady=5)
text_frame.pack(side='left', fill='both', expand=True, padx=5, pady=5)

# Create buttons to open download folders
video_button = tk.Button(video_frame, text='Open Videos Folder', command=lambda: open_download_folder('videos'), bg='#3498db', fg='white', relief='flat')
video_button.pack(pady=10)

images_button = tk.Button(images_frame, text='Open Images Folder', command=lambda: open_download_folder('images'), bg='#3498db', fg='white', relief='flat')
images_button.pack(pady=10)

text_button = tk.Button(text_frame, text='Open Text Folder', command=lambda: open_download_folder('text'), bg='#3498db', fg='white', relief='flat')
text_button.pack(pady=10)

# Add counters for session downloads
video_counter_label = tk.Label(video_frame, text=f"Session Videos: {session_video_count}", bg='#ecf0f1')
images_counter_label = tk.Label(images_frame, text=f"Session Images: {session_images_count}", bg='#ecf0f1')
text_counter_label = tk.Label(text_frame, text=f"Session Text: {session_text_count}", bg='#ecf0f1')

video_counter_label.pack(pady=10)
images_counter_label.pack(pady=10)
text_counter_label.pack(pady=10)

# Add a counter for total links in the Excel file
total_links_counter_label = tk.Label(toolbar_frame, text="Total Links: 0", bg='#ecf0f1')
total_links_counter_label.pack(side='left', padx=10)

root.mainloop()
